In [2]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision.models import ResNet18_Weights
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from torchvision import datasets
from sklearn.metrics import accuracy_score
import timm

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [4]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(60),
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [5]:
dataset = datasets.ImageFolder(r'C:\Users\Saumya\PycharmProjects\pythonProject6\data\AugmentedAlzheimerDataset',transform=train_transform)


In [6]:
class_counts = {}
subset_indices = []

for idx, (_, label) in enumerate(dataset.samples):  
    if class_counts.get(label, 0) < 4000:  
        subset_indices.append(idx)
        class_counts[label] = class_counts.get(label, 0) + 1

# Create a subset dataset
subset_dataset = Subset(dataset, subset_indices)


train_size = int(0.8 * len(subset_indices))  # 80% train
test_size = len(subset_indices) - train_size  # 20% test

# Split the subset dataset
train_dataset, test_dataset = random_split(subset_dataset, [train_size, test_size])
train_val_split = int(0.8 * train_size)  
val_size = train_size - train_val_split
train_dataset, val_dataset = random_split(train_dataset, [train_val_split, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [7]:
testing_dataset = datasets.ImageFolder(r'C:\Users\Saumya\PycharmProjects\pythonProject6\OriginalDataset',transform=test_transform)

In [8]:
final_test_loader = DataLoader(testing_dataset, batch_size=32, shuffle=False)

In [9]:
class_mapping = {
    "Mild Impairment":"MildDemented",
    "Moderate Impairment":"ModeratedDemented",
    "No Impairment":"NonDemented",
    "Very Mild Impairment":"VeryMildDemented"
}

In [10]:
new_test_dataset = datasets.ImageFolder(r"C:\Users\Saumya\PycharmProjects\pythonProject6\test",transform=test_transform)
new_test_dataset.class_to_idx = {class_mapping[c]: idx for c, idx in new_test_dataset.class_to_idx.items() if c in class_mapping}

In [11]:
new_test_loader = DataLoader(new_test_dataset, batch_size=32, shuffle=False)

In [12]:
from timm import create_model

model = create_model('convnextv2_nano', pretrained=True)
print(model)

ConvNeXt(
  (stem): Sequential(
    (0): Conv2d(3, 80, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((80,), eps=1e-06, elementwise_affine=True)
  )
  (stages): Sequential(
    (0): ConvNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (conv_dw): Conv2d(80, 80, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=80)
          (norm): LayerNorm2d((80,), eps=1e-06, elementwise_affine=True)
          (mlp): GlobalResponseNormMlp(
            (fc1): Conv2d(80, 320, kernel_size=(1, 1), stride=(1, 1))
            (act): GELU()
            (drop1): Dropout(p=0.0, inplace=False)
            (grn): GlobalResponseNorm()
            (fc2): Conv2d(320, 80, kernel_size=(1, 1), stride=(1, 1))
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (shortcut): Identity()
          (drop_path): Identity()
        )
        (1): ConvNeXtBlock(
          (conv_dw): Conv2d(80, 80, kernel_size=(7, 7), stride=(1, 1), 

In [13]:
print(model)

ConvNeXt(
  (stem): Sequential(
    (0): Conv2d(3, 80, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((80,), eps=1e-06, elementwise_affine=True)
  )
  (stages): Sequential(
    (0): ConvNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (conv_dw): Conv2d(80, 80, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=80)
          (norm): LayerNorm2d((80,), eps=1e-06, elementwise_affine=True)
          (mlp): GlobalResponseNormMlp(
            (fc1): Conv2d(80, 320, kernel_size=(1, 1), stride=(1, 1))
            (act): GELU()
            (drop1): Dropout(p=0.0, inplace=False)
            (grn): GlobalResponseNorm()
            (fc2): Conv2d(320, 80, kernel_size=(1, 1), stride=(1, 1))
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (shortcut): Identity()
          (drop_path): Identity()
        )
        (1): ConvNeXtBlock(
          (conv_dw): Conv2d(80, 80, kernel_size=(7, 7), stride=(1, 1), 

In [14]:
old_head = model.head
num_features = model.head.fc.out_features  
num_classes = len(dataset.classes)
model.head = nn.Sequential(
    old_head,
    nn.Linear(num_features, num_classes)
)
model = model.to(device)

In [15]:
print(model)

ConvNeXt(
  (stem): Sequential(
    (0): Conv2d(3, 80, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((80,), eps=1e-06, elementwise_affine=True)
  )
  (stages): Sequential(
    (0): ConvNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (conv_dw): Conv2d(80, 80, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=80)
          (norm): LayerNorm2d((80,), eps=1e-06, elementwise_affine=True)
          (mlp): GlobalResponseNormMlp(
            (fc1): Conv2d(80, 320, kernel_size=(1, 1), stride=(1, 1))
            (act): GELU()
            (drop1): Dropout(p=0.0, inplace=False)
            (grn): GlobalResponseNorm()
            (fc2): Conv2d(320, 80, kernel_size=(1, 1), stride=(1, 1))
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (shortcut): Identity()
          (drop_path): Identity()
        )
        (1): ConvNeXtBlock(
          (conv_dw): Conv2d(80, 80, kernel_size=(7, 7), stride=(1, 1), 

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [17]:
from sklearn.metrics import accuracy_score

def evaluate_model(model, val_loader, device):
    model.eval()  # Set model to evaluation mode
    all_labels = []
    all_preds = []

    with torch.no_grad():  # No need to track gradients
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)  # Move to correct device
            outputs = model(images)  # Get model predictions
            _, preds = torch.max(outputs, 1)  # Get predicted class index

            all_labels.extend(labels.cpu().numpy())  # Store true labels
            all_preds.extend(preds.cpu().numpy())  # Store predictions

    accuracy = accuracy_score(all_labels, all_preds) * 100  
    print(f"Test Accuracy: {accuracy:.2f}%")
    
    return accuracy  
def train_model(model, train_loader,val_loader, optimizer, criterion, num_epochs, device):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
    
    
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
    
            optimizer.zero_grad()  # Clear gradients
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights
            
            train_loss += loss.item()
            
        train_loss /= len(train_loader)

    
    
        val_acc=evaluate_model(model, val_loader, device)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Accuracy: {val_acc:.2f}%")


In [18]:
for param in model.parameters():
    param.requires_grad = True

In [19]:
print("\n Training only the classifier (Feature Extraction Mode)...")
train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs=50,device=device)


 Training only the classifier (Feature Extraction Mode)...
Test Accuracy: 70.51%
Epoch 1/50, Loss: 0.7153, Accuracy: 70.51%
Test Accuracy: 83.32%
Epoch 2/50, Loss: 0.4356, Accuracy: 83.32%
Test Accuracy: 90.62%
Epoch 3/50, Loss: 0.1850, Accuracy: 90.62%
Test Accuracy: 92.23%
Epoch 4/50, Loss: 0.1487, Accuracy: 92.23%
Test Accuracy: 94.26%
Epoch 5/50, Loss: 0.8054, Accuracy: 94.26%
Test Accuracy: 94.26%
Epoch 6/50, Loss: 0.3446, Accuracy: 94.26%
Test Accuracy: 96.91%
Epoch 7/50, Loss: 0.0607, Accuracy: 96.91%
Test Accuracy: 96.02%
Epoch 8/50, Loss: 0.0764, Accuracy: 96.02%
Test Accuracy: 96.29%
Epoch 9/50, Loss: 0.1322, Accuracy: 96.29%
Test Accuracy: 97.89%
Epoch 10/50, Loss: 0.0513, Accuracy: 97.89%
Test Accuracy: 96.76%
Epoch 11/50, Loss: 0.0578, Accuracy: 96.76%
Test Accuracy: 98.55%
Epoch 12/50, Loss: 0.0101, Accuracy: 98.55%
Test Accuracy: 97.27%
Epoch 13/50, Loss: 0.0595, Accuracy: 97.27%
Test Accuracy: 97.38%
Epoch 14/50, Loss: 0.0052, Accuracy: 97.38%
Test Accuracy: 98.24%
Epo

KeyboardInterrupt: 

In [20]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-6)
print("\n Training only the classifier (Feature Extraction Mode)...")
train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs=7,device=device)


 Training only the classifier (Feature Extraction Mode)...
Test Accuracy: 99.14%
Epoch 1/7, Loss: 0.0018, Accuracy: 99.14%
Test Accuracy: 99.30%
Epoch 2/7, Loss: 0.0005, Accuracy: 99.30%
Test Accuracy: 99.41%
Epoch 3/7, Loss: 0.0006, Accuracy: 99.41%
Test Accuracy: 99.41%
Epoch 4/7, Loss: 0.0001, Accuracy: 99.41%
Test Accuracy: 99.34%
Epoch 5/7, Loss: 0.0002, Accuracy: 99.34%
Test Accuracy: 99.57%
Epoch 6/7, Loss: 0.0001, Accuracy: 99.57%
Test Accuracy: 99.45%
Epoch 7/7, Loss: 0.0001, Accuracy: 99.45%


In [23]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, precision_score, recall_score

model.eval()    
all_labels = []
all_preds = []
val_loss = 0.0

with torch.no_grad():  # No need to track gradients for testing
    for images, labels in final_test_loader:
        images, labels = images.to(device), labels.to(device)  # Move labels and images to the current device
        outputs = model(images)  # Get model predictions
        _, preds = torch.max(outputs, 1)  # Get predicted class index (highest probability)
        loss = criterion(outputs, labels)
        val_loss += loss.item()
        all_labels.extend(labels.cpu().numpy())  # Store true labels
        all_preds.extend(preds.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(all_labels, all_preds)

# Compute F1-score
f1 = f1_score(all_labels, all_preds, average="weighted")  # Use "macro" for equal class weighting

precision = precision_score(all_labels, all_preds, average="weighted")  # Use "micro", "macro", or "weighted"

# Compute recall
recall = recall_score(all_labels, all_preds, average="weighted")
# Print results
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Test Loss: {val_loss:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

# P
print("\nClassification Report:")
print(classification_report(all_labels, all_preds))


Test Accuracy: 99.59%
Test Loss: 2.4963
F1 Score: 0.9959
Precision: 0.9959
Recall: 0.9959

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       896
           1       1.00      1.00      1.00        64
           2       1.00      1.00      1.00      3200
           3       0.99      1.00      0.99      2240

    accuracy                           1.00      6400
   macro avg       1.00      1.00      1.00      6400
weighted avg       1.00      1.00      1.00      6400



In [25]:
torch.save(model,"convextv2_nano_finetuned.pth")